In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import sys, os
sys.path.append('../')
from data_handling import Container,load_data,Normalize,Denormalize
from plot_style.style_prab import load_preset,figsize,cmap_nicify
from plot_style.style_prab import colors as colors_preset
from train_ensemble import train, NN, Loss
from analysis.scan_model import median,quantile
load_preset(scale=1,font_path='../plot_style/font')
import pickle
mymap = cmap_nicify(cmap='YlGnBu_r',idx_white=1,size_white=50)
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches


def plot(x,y,ax,color,label=None,datalabel='',pos='left',rot=0):
    
    mean = y[0]
    above = y[1]
    below = y[2]
    #c2 = (color[0]+0.1,color[1]+0.1,color[2]+0.1,color[3])
    ax.fill_between(x,above,below,alpha=0.5,color=color,linewidth=0)
    p1 = ax.plot(x,mean,color=color,alpha=1,label=datalabel)
    yl = ax.set_ylabel(label)
    ax.yaxis.set_label_position("right")
    yl.set_rotation(rot)

    return p1

In [ ]:
import pickle
from ensemble.Bregressor_ import BaggingRegressor
from ensemble.utils import io

model_path = '../models/'
with open(f'{model_path}config.pkl', 'rb') as fp:
    config = pickle.load(fp)
    norm = config['norm']


if isinstance(config['model'],list):
    model = nn.ModuleList(config['model'])

    ensemble = BaggingRegressor(estimator=model,               
                                n_estimators=config['estimators'],
                                cuda=False,)
    ensemble.set_criterion(config['loss_fun'])
    io.load(ensemble, model_path)
else:
    model = config['model']
    ensemble = BaggingRegressor_te(estimator=model,               
                                n_estimators=config['estimators'],
                                cuda=False,)
    ensemble.set_criterion(config['loss_fun'])
    io_te.load(ensemble, model_path)
    
    
path='../data/dataframe_combined_espec_interpolated_gaia_energy_2022.h5'
trainset, testset, _ = load_data(
config["path"],
config["inputs"],
config["outputs"],
samples=config['samples'],
ratio=config['ratio'],
start_ind=0,
norm=norm,
random=False,
)

trainset = Container(trainset)
testset = Container(testset)

testset.x = testset.x.to('cpu')
testset.y = testset.y.to('cpu')
ydata = Denormalize(testset.y.detach().numpy(),norm[1])


In [ ]:
x_ = torch.linspace(-2.5,2.5,100)
x = torch.zeros(100,14)
x[:,8] = x_

z = median(x,ensemble).detach().numpy()
z = Denormalize(z,norm[1])
z = z.reshape(100,5)

c_zfoc = ((162*1e3)/6.9)
x = Denormalize(x,norm[0])
x = x.reshape(100,14)[:,8].detach().numpy()
x = (x-x[50])*c_zfoc

In [ ]:
with open('../data/waterfall_data_xrays.pkl', 'rb') as handle:
    wfp = pickle.load(handle)

In [ ]:
config['outputs']

In [ ]:
fig_w = figsize['inch']['double_column_width']
fig,axs = plt.subplots(1,3, figsize=(fig_w,fig_w*0.3))
ax=axs[[0,1,2]]
color = colors_preset[1]

#z[10:-10,0] = z[10:-10,0] + 20

extent = [227,290,0,17]
ax[0].plot(z[10:-10,0].flatten(),z[10:-10,-2].flatten(),'.-',lw=0.5,mew=0,ms=4,color=color,label='Explanatory model')
ax[0].pcolormesh(wfp['E'],wfp['wl'],wfp['wfp'].T,cmap=mymap)


ax[0].set_xlim(extent[0:2])
ax[0].set_ylim(extent[-2:])
ax[0].set_xlabel('E (MeV)')
ax[0].set_ylabel(r'$\lambda_{xray}$ (nm)')

extent = [227,290,9,15]
ax[1].plot(z[10:-10,0].flatten(),z[10:-10,-2].flatten(),'.-',lw=0.5,mew=0,ms=4,color=color,label='Explanatory model')
ax[1].hexbin(ydata[:,0].flatten(),ydata[:,-2].flatten(),extent=extent,bins=15,cmap = 'bone_r',
             lw=0.05,gridsize=[50,int(50*2./4)]);


ax[1].set_xlim(extent[0:2])
ax[1].set_ylim(extent[-2:])
ax[1].set_xlabel('E (MeV)')
ax[1].set_ylabel(r'$\lambda_{xray}$ (nm)')


extent = [0.005,0.07,0.1,0.25]#[1,20,1.4,3]
ax[2].plot(z[10:-10,1].flatten()/z[10:-10,0].flatten(),z[10:-10,-1].flatten()/z[10:-10,-2].flatten(),'.-',lw=0.5,mew=0,ms=4,color=color,label='Infered model')
ax[2].hexbin(ydata[:,1].flatten()/ydata[:,0].flatten(),ydata[:,-1].flatten()/ydata[:,-2].flatten(),extent=extent,bins=15,
             cmap = 'bone_r',lw=0.05,gridsize=[50,int(50*2./4)]);
#ax[1].set_xlim(extent[0:2])
#ax[1].set_ylim(extent[-2:])
ax[2].set_xlabel(r'$\Delta$E/E ')
ax[2].set_ylabel(r'$\Delta\lambda_{xray}/\lambda_{xray}$')
ax[2].yaxis.tick_right()
ax[2].yaxis.set_label_position("right")

wl_u = 5e6 # nm
K = 0.29 #0.29 # unitless
mec2 = 0.511 #MeV 

def xray_wl(E):
    return (wl_u*mec2**2/(2*(E)**2))*(1+K**2/2) #+ (np.pi*2e-7)*(E)**2/(mec2)**2)




E = np.linspace(230,285)
xwl = xray_wl(E)
ax[0].plot(E,xwl,'k',label='Undulator eq.')


dE = np.linspace(0.005,0.05)
#ax[2].plot(dE,dE*2/1.5+0.15,'k',label='Undulator eq.')
#ax[2].text(0.5, 0.95, f'slope={2/1.2:0.3}', transform=ax[2].transAxes,verticalalignment='top')


ax[0].legend(frameon=False,loc='upper right', bbox_to_anchor=(0.75, 0.5))
#ax[1].text(0.55, 0.95, r'$z_{foc}=$'+f'{int(x[10])}'+r'$\mu m$', transform=ax[1].transAxes,verticalalignment='top')
#ax[1].text(0.55, 0.3, r'$z_{foc}=$'+f'{int(x[-10])}'+r'$\mu m$', transform=ax[1].transAxes,verticalalignment='top')

ax[1].text(0.65, 0.5, r'$z_{foc}=$'+f'{int(x[-10])}'+r'$\mu m$', transform=ax[1].transAxes,verticalalignment='top',rotation=-45)
ax[1].text(0.1, 0.8, r'$z_{foc}=$'+f'{int(x[10])}'+r'$\mu m$', transform=ax[1].transAxes,verticalalignment='top',rotation=-14)

ax[0].text(0.03, 0.95, r'(a)', transform=ax[0].transAxes,verticalalignment='top')
#ax[1].text(0.03, 0.95, r'(c)', transform=ax[1].transAxes,verticalalignment='top')
ax[1].text(0.03, 0.95, r'(b)', transform=ax[1].transAxes,verticalalignment='top')
ax[2].text(0.03, 0.95, r'(c)', transform=ax[2].transAxes,verticalalignment='top')


ax[1].annotate("", xy=(285,10), xytext=(275,11),
            arrowprops=dict(arrowstyle="->",linewidth=0.5))
ax[1].annotate("", xytext=(230,13.3), xy=(245,13),
            arrowprops=dict(arrowstyle="->",linewidth=0.5))

ax[2].annotate("", xy=(0.04,0.205), xytext=(0.05,0.22),
            arrowprops=dict(arrowstyle="->",linewidth=0.5))
ax[2].annotate("", xytext=(0.06,0.184), xy=(0.07,0.190),
            arrowprops=dict(arrowstyle="->",linewidth=0.5))



plt.savefig('figures/corr_electron-xrays.png',dpi=300,)#bbox='tight')

In [ ]:
def bw(E,dE):
    lu = 5e-3
    K = 0.29
    alpha = (lu/2)*(1+K**2/2)
    gamma2 = (E/0.511+1)**2
    l_shift = 2e-9 + (5/2)*1e-9
    C = l_shift*gamma2/alpha
    Nu = 100
    return np.sqrt((1/Nu)**2+((2/(1+C))*dE/E)**2)


lambda_corr = 2e-9
div_corr = (5/2)*1e-9
ltot = lambda_corr + div_corr
gamma2 = (260/0.5+1)**2
gamma2*ltot/(2.6e-3)

E = z[10:-10,0].flatten()
dE = z[10:-10,1].flatten() #np.linspace(0.005,0.05,80)
dl_l = bw(E,dE)

ax[2].plot(dE/E,dl_l)
fig